In [34]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import scipy.special as sp
import IGM

r0 = 2.818e-13 # classical electron radius in cm
m_e = 9.10938e-28 # electron mass in gram
c = 2.998e10 # speed of light in cm/s
h = 6.6261e-27 # planck constant in cm^2 g s-1
eV_to_erg = 1.6022e-12
parsec_to_cm = 3.085677581e18 # cm per parsec

kB = 1.3808e-16 # boltzmann constant in cm^2 g s^-2 K^-1
T_CMB = 2.725 # CMB temperature today in K

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_p = 1.67262193269e-24 # proton mass in g
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium mass fraction
f_H = 0.76 # hydrogen fraction
Y_He = 0.24

E_e = m_e * c * c
rho_crit = 3 * H_0 * H_0 / 8 / np.pi / G
e = 4.80326e-10 # electron charge in esu

z = 2
z2 = (1 + z) / 3
T_CMB *= 1 + z 

T_IGM = 9245.6 # IGM temperature at z = 2 in K

In [35]:
omega_b = omega_b_0 * (1 + z)**3
mean_n_b = 3 * omega_b * H_0 * H_0 / (8 * np.pi * G * m_p) # mean number density of baryons at the redshift
n_b = mean_n_b # for mean density
    
n_H = n_b * (1 - Y_He) # nuclei in hydrogen
n_He = n_H * f_He # nuclei in helium, assume He is all 4He

n_e = n_H + 2 * n_He # for fully ionized

In [36]:
omega_p = c * np.sqrt(4 * np.pi * r0 * n_e)

k = omega_p / c * np.logspace(np.log10(1.000001), np.log10(2), 100) # wave number
theta_deg = np.arange(5, 360, 5)
theta_rad = np.radians(theta_deg)

In [37]:
print(omega_p/c, k[0], theta_rad[0])

4.6007454504527044e-09 4.6007500511981544e-09 0.08726646259971647


In [38]:
n = n_e + n_H + n_He

B = np.sqrt(8 * np.pi * kB * T_IGM * n) # equipartion field

omega_c = e * B / (m_e * c)

In [39]:
def dispersion(omega,k,theta_rad):
    res = omega**2 - omega_p**2 / (1 - omega_c**2 / omega**2) - c**2 * k**2 * np.cos(theta_rad)**2 - 1 / (1 - omega_c**2 / omega**2) * omega_c**2 * omega_p**4 / ((omega**2 - c**2 * k**2) * (omega**2 - omega_c**2) - omega_p**2 * omega**2) - c**4 * k**4 * np.sin(theta_rad)**2 * np.cos(theta_rad)**2 / (omega**2 - omega_p**2 - c**2 * k**2 * np.sin(theta_rad)**2)
    return res

In [40]:
# grid has shape (len(k), len(angle)), same row same k, same col same angle
omega = np.zeros((len(k), len(theta_rad)))

for i in range(len(k)):
    for j in range(len(theta_rad)):
        dispersion_fixed = lambda omega: dispersion(omega, k[i], theta_rad[j])
        initial_guess = omega_p/10
        omega[i][j] = fsolve(dispersion_fixed, initial_guess)[0]

In [41]:
test = np.zeros((len(k), len(theta_rad)))
for i in range(len(k)):
    for j in range(len(theta_rad)):
        test[i][j] = dispersion(omega[i][j], k[i], theta_rad[j])
        
print(np.min(test), np.max(test))

-1.1898135376497976e-05 3.013859668499208e-06


In [42]:
def get_eigen(omega, k, theta_rad):
    zeta_x = np.array([np.sin(theta_rad), 0, np.cos(theta_rad)]) # for now, as zeta will be canceled out later
    zeta_y = 1j * omega_c * omega_p**2 * omega / ((omega**2 - c**2 * k**2) * (omega**2 - omega_c**2) - omega_p**2 * omega**2) * zeta_x
    zeta_z = -c**2 * k**2 * np.sin(theta_rad) * np.cos(theta_rad) / (omega**2 - omega_p**2 - c**2 * k**2 * np.sin(theta_rad)**2) * zeta_x
    return zeta_x, zeta_y, zeta_z

In [43]:
def get_D(i, j):
    omega_ij = omega[i][j]
    zeta_x, zeta_y, zeta_z = get_eigen(omega_ij, k[i], theta_rad[j])
    mag2_x = zeta_x[0]**2 + zeta_x[1]**2 + zeta_x[2]**2
    mag2_y = np.abs(zeta_y[0])**2 + np.abs(zeta_y[1])**2 + np.abs(zeta_y[2])**2
    mag2_z = zeta_z[0]**2 + zeta_z[1]**2 + zeta_z[2]**2
    mag2_zeta = mag2_x + mag2_y + mag2_z
    
    add1 = 2 * omega_ij * mag2_zeta
    co1 = omega_p**2 * omega_c**2 / (omega_ij**3 * (1 - omega_c**2 / omega_ij**2)**2)
    co2 = 2 * (mag2_x + mag2_y) - 1j * (np.dot(np.conj(zeta_x), zeta_y) - np.dot(np.conj(zeta_y), zeta_x)) / omega_c * omega_ij * (1 + omega_c**2 / omega_ij**2)
    return add1 + co1 * co2.real # co2 is always xxx + 0j, only real part make sense

In [44]:
D = np.zeros((len(k), len(theta_rad)))
for i in range(len(k)):
    for j in range(len(theta_rad)):
        D[i][j] = get_D(i, j)

In [13]:
IGM_00 = IGM.IGM_N(0,0) # mean density
n_CRe = IGM_00.back_Euler() # number density of CRe, shape (399, 399)
E = IGM.E
gamma = E / E_e + 1

In [21]:
print(IGM_00.z[240])
n_CRe_z2 = n_CRe[:, 240] # number density of CRe at z = 2, shape (399,)
n_CRe_z2 = np.insert(n_CRe_z2, 0, 0) # insert 0 at beginning for calculation, shape (400, )

2.0083681725319145


In [18]:
p = gamma * m_e * c # shape (400, )
omega_s = omega_c / gamma # shape (400, )

In [72]:
def get_n_Bessel(omega, k_para, omega_s):
    # cos(alpha) = (omega - n * omega_s) / k_parallel / v
    n_b1 = (omega - k_para * c) / omega_s
    n_b2 = (omega + k_para * c) / omega_s
    
    lower_bound = int(min(n_b1, n_b2)) + 1
    upper_bound = int(max(n_b1, n_b2))
    
    #print((omega - n_b1 * omega_s) / k_para / c)
    #print((omega - n_b2 * omega_s) / k_para / c)
    
    return np.array(range(lower_bound, upper_bound + 1))

In [62]:
print(get_n_Bessel(omega[0][0], k[0]*np.cos(theta_rad[0]), omega_s[0]))

0.9999999999999999
-0.9999999999999998
[170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331
 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349
 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367
 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385
 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403
 404 405 406

In [74]:
def get_Bessel(omega, n_Bessel, omega_s, k_para, k_orth, k, theta_rad):
    cosa = (omega - n_Bessel * omega_s) / k_para / c
    sina = np.sqrt(1 - cosa**2)
    
    z_Bessel = k_orth * c * sina / omega_s
    Jnz = sp.jv(n_Bessel, z_Bessel)
    Jnz_p = sp.jvp(n_Bessel, z_Bessel, n=1)
    
    zeta_x, zeta_y, zeta_z = get_eigen(omega, k, theta_rad)
    
    res_v = n_Bessel / z_Bessel * Jnz * zeta_x * sina + Jnz * zeta_z * cosa + Jnz_p * zeta_y / 1j * sina
    #print(res_v)
    res = res_v[0].real**2 + res_v[1].real**2 + res_v[2].real**2
    #print(res)
    
    return res

In [53]:
get_Bessel(omega[0][0], 200, omega_s[0], k[0]*np.cos(theta_rad[0]), k[0]*np.sin(theta_rad[0]), k[0], theta_rad[0])

[-1.02273043e-212+0.j  0.00000000e+000+0.j -1.16898623e-211+0.j]
0.0


0.0

In [79]:
def get_int(i, j):
    k_para = k[i] * np.cos(theta_rad[j])
    k_orth = k[i] * np.sin(theta_rad[j])
    
    int_sum = 0
    for pi in range(len(p)-1):
        Delta = omega_s[pi] / (np.abs(k_para) * c)
        co1 = p[pi]**2 * (p[pi+1] - p[pi]) * (n_CRe_z2[pi+1] - n_CRe_z2[pi]) * m_e * c * Delta / omega_s[pi]
        
        n_Bessel = get_n_Bessel(omega[i][j], k_para, omega_s[pi])
        print(pi, n_Bessel.shape)
        sum_Bessel = sum(get_Bessel(omega[i][j], ni_Bessel, omega_s[pi], k_para, k_orth, k[i], theta_rad[j]) for ni_Bessel in n_Bessel)
        int_sum += (co1 * sum_Bessel)
        
    return int_sum

In [57]:
def get_damp_rate(i, j):
    res = 2 * np.pi**2 * omega_p**2 * omega[i][j] / (D[i][j] * n_e) * get_int(i,j)
    return res

In [89]:
import time

k = omega_p / c * np.array([1.01, 1.05, 1.1, 1.5, 2]) # wave number
theta_deg = np.array([5, 45, 85])
theta_rad = np.radians(theta_deg)

damp_rate = np.zeros((len(k), len(theta_rad)))
time_start = time.time()
for j in range(len(theta_rad)):
    for i in range(len(k)):
        print(f'i = {i}, j = {j}, {(time.time() - time_start)/60}')
        damp_rate[i][j] = get_damp_rate(i, j)
        np.save('damp_rate.npy', damp_rate)

i = 0, j = 0, 3.2862027486165364e-06
0 (819,)
1 (819,)
2 (819,)
3 (819,)
4 (819,)
5 (819,)
6 (819,)
7 (819,)
8 (819,)
9 (819,)
10 (819,)
11 (819,)
12 (819,)
13 (819,)
14 (819,)
15 (819,)
16 (819,)
17 (819,)
18 (819,)
19 (819,)
20 (819,)
21 (819,)
22 (819,)
23 (819,)
24 (819,)
25 (819,)
26 (819,)
27 (819,)
28 (819,)
29 (819,)
30 (819,)
31 (819,)
32 (819,)
33 (819,)
34 (819,)
35 (819,)
36 (819,)
37 (819,)
38 (819,)
39 (819,)
40 (819,)
41 (819,)
42 (819,)
43 (819,)
44 (819,)
45 (819,)
46 (819,)
47 (820,)
48 (820,)
49 (820,)
50 (820,)
51 (820,)
52 (820,)
53 (820,)
54 (820,)
55 (820,)
56 (820,)
57 (820,)
58 (820,)
59 (820,)
60 (820,)
61 (820,)
62 (820,)
63 (820,)
64 (820,)
65 (820,)
66 (820,)
67 (820,)
68 (820,)
69 (820,)
70 (820,)
71 (820,)
72 (820,)
73 (820,)
74 (820,)
75 (820,)
76 (820,)
77 (820,)
78 (820,)
79 (820,)
80 (820,)
81 (820,)
82 (820,)
83 (820,)
84 (820,)
85 (820,)
86 (820,)
87 (820,)
88 (820,)
89 (820,)
90 (820,)
91 (820,)
92 (820,)
93 (820,)
94 (820,)
95 (820,)
96 (820,)
97 

In [90]:
#k = omega_p / c * np.array([1.01, 1.05, 1.1, 1.5, 2]) # wave number
#theta_deg = np.array([5, 45, 85])
#theta_rad = np.radians(theta_deg)

print(damp_rate)

[[ 0.00000000e+000  2.32297629e-193  1.19549235e-162]
 [ 0.00000000e+000  1.05608374e-177  7.86168961e-151]
 [ 0.00000000e+000  1.44430056e-159  8.97123559e-137]
 [-1.39545934e-073 -2.33612801e-070 -2.12202420e-072]
 [-5.26680554e-073 -4.73840378e-072 -8.96663304e-073]]
